In [1]:
# Making imports convenient
import sys
import os
PATH=os.getcwd().split('/notebooks')[0]
sys.path.insert(1, PATH)

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset,concatenate_datasets
import transformers
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

from transformers import AutoTokenizer, DataCollatorWithPadding,AutoModelForSequenceClassification,AdamW,get_scheduler,TrainingArguments,Trainer,EarlyStoppingCallback
from sklearn.model_selection import ParameterGrid
from src.utils.myutils import *
import yaml
import json

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

CS_DATA_PATH = PATH + '/data/CS/processed/BABE/train.csv'
CONFIG_PATH = PATH + '/src/utils/config.yaml'

BATCH_SIZE = 16
transformers.utils.logging.set_verbosity_error()

In [7]:
data_babe = load_dataset('csv',data_files = PATH + '/data/CS/processed/BABE/train.csv')['train']
data_cwnc = load_dataset('csv',data_files = PATH + '/data/CS/processed/CWNC/train.csv')['train']

with open(CONFIG_PATH) as f:
    config_data = yaml.load(f, Loader=yaml.FullLoader)
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

Using custom data configuration default-f28f8af5b44ab214
Reusing dataset csv (/home/horyctom/.cache/huggingface/datasets/csv/default-f28f8af5b44ab214/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Using custom data configuration default-6f7c93c8687304cb
Reusing dataset csv (/home/horyctom/.cache/huggingface/datasets/csv/default-6f7c93c8687304cb/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)


In [8]:
model_name = 'ufal/robeczech-base'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False,padding=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

babe_tok = preprocess_data(data_babe,tokenizer,'sentence')
cwnc_tok = preprocess_data(data_cwnc,tokenizer,'sentence')

Loading cached processed dataset at /home/horyctom/.cache/huggingface/datasets/csv/default-f28f8af5b44ab214/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-74b152a0ca0f6626.arrow
Loading cached processed dataset at /home/horyctom/.cache/huggingface/datasets/csv/default-6f7c93c8687304cb/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff/cache-5c95220f8fd21e5c.arrow


In [14]:
training_args = TrainingArguments(
            output_dir = './',
            num_train_epochs=3,
            save_total_limit=2,
            disable_tqdm=False,
            per_device_train_batch_size=16,  
            warmup_steps=0,
            weight_decay=0.1,
            learning_rate=2e-5)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


## BABE

In [15]:
scores = []
for train_index, val_index in skfold.split(babe_tok['input_ids'],babe_tok['label']):

    token_train = Dataset.from_dict(babe_tok[train_index])
    token_valid = Dataset.from_dict(babe_tok[val_index])


    model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
    model.to(device)
    trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,tokenizer=tokenizer)
    trainer.train()

    #evaluation
    eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
    scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
        
print(scores)
print(np.mean(scores))

/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcd

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb448cab0851f27c5104.ae62083e57028e6866dba352dfd4261396c2f0e8978f299e3a17c055c564de09
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "ge

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb448cab0851f27c5104.ae62083e57028e6866dba352dfd4261396c2f0e8978f299e3a17c055c564de09
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "ge

...

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb448cab0851f27c5104.ae62083e57028e6866dba352dfd4261396c2f0e8978f299e3a17c055c564de09
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "ge

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




[0.7872, 0.7808, 0.7868589743589743, 0.7884615384615384, 0.7724358974358975]
0.783151282051282


## CWNC

In [16]:
scores = []
for train_index, val_index in skfold.split(cwnc_tok['input_ids'],cwnc_tok['label']):

    token_train = Dataset.from_dict(cwnc_tok[train_index])
    token_valid = Dataset.from_dict(cwnc_tok[val_index])


    model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
    model.to(device)
    trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,tokenizer=tokenizer)
    trainer.train()

    #evaluation
    eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
    scores.append(compute_metrics(model,device,eval_dataloader)['f1'])
        
print(scores)
print(np.mean(scores))

/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcd

Step,Training Loss
500,0.575000


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./checkpoint-500/tokenizer_config.json
Special tokens file saved in ./checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb448ca

Step,Training Loss
500,0.515300


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./checkpoint-500/tokenizer_config.json
Special tokens file saved in ./checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb448ca

...

Step,Training Loss
500,0.487900


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./checkpoint-500/tokenizer_config.json
Special tokens file saved in ./checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb448ca

Step,Training Loss
500,0.697200


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./checkpoint-500/tokenizer_config.json
Special tokens file saved in ./checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




[0.7331319234642497, 0.7492447129909365, 0.7368951612903226, 0.7620967741935484, 0.7268145161290324]
0.741636617613618


## Trained on BABE evaluated on CWNC

In [17]:
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
model.to(device)
trainer = Trainer(model,training_args,train_dataset=babe_tok,data_collator=data_collator,tokenizer=tokenizer)
trainer.train()

#evaluation
eval_dataloader = DataLoader(cwnc_tok, batch_size=BATCH_SIZE, collate_fn=data_collator)
print(compute_metrics(model,device,eval_dataloader)['f1'])

loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb448cab0851f27c5104.ae62083e57028e6866dba352dfd4261396c2f0e8978f299e3a17c055c564de09
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 51961
}

loading weights file https://huggingface.co/ufal/robeczech-base/resolve/m

Step,Training Loss
500,0.433400


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./checkpoint-500/tokenizer_config.json
Special tokens file saved in ./checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




0.5276098347440548


## Trained on CWNC evaluated on BABE

In [18]:
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=2);
model.to(device)
trainer = Trainer(model,training_args,train_dataset=cwnc_tok,data_collator=data_collator,tokenizer=tokenizer)
trainer.train()

#evaluation
eval_dataloader = DataLoader(babe_tok, batch_size=BATCH_SIZE, collate_fn=data_collator)
print(compute_metrics(model,device,eval_dataloader)['f1'])

loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb448cab0851f27c5104.ae62083e57028e6866dba352dfd4261396c2f0e8978f299e3a17c055c564de09
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 51961
}

loading weights file https://huggingface.co/ufal/robeczech-base/resolve/m

Step,Training Loss
500,0.480700


Saving model checkpoint to ./checkpoint-500
Configuration saved in ./checkpoint-500/config.json
Model weights saved in ./checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./checkpoint-500/tokenizer_config.json
Special tokens file saved in ./checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




0.6665598975016015
